In [1]:
import pandas as pd
import numpy as np
import choix
from sklearn.preprocessing import normalize
# get data through terminal: wget https://raw.githubusercontent.com/kpelechrinis/BT-model/master/nba1617.csv

In [27]:
# An example of Bradley-Terry model with win-loss relationships in the NBA

df = pd.read_csv("nba1617.csv") # : a file with the game results from the 2016-17 NBA season
teams = list(set(df.Home.unique()))
t = list(np.sort(teams))

data = []

for i in range(len(df.Visitor)):
    if df.PTSV[i] > df.PTSH[i]: # if visitor win
        el = (t.index(df.Visitor[i]),t.index(df.Home[i])) # set vistor first in tuple 
    else:
        el = (t.index(df.Home[i]),t.index(df.Visitor[i])) # else, set home first in tuple
    data.append(el)

skills = choix.ilsr_pairwise(30, data) # pairwise comparison

# ------------------------------

prob_mat = np.zeros([30]) # create first empty array

for i in range(30):
    win_probs = []
    for j in range(30):
        win_probs.append(choix.probabilities([i, j], skills)[0])
        
    if prob_mat.sum() == 0:
        prob_mat = prob_mat + win_probs #first iteration, replace zero-array
        
    else:
        prob_mat = np.vstack((prob_mat,np.array(win_probs))) #concurrent iteration, append array as row-vector, creating the propability matrix 

prob_df = pd.DataFrame(prob_mat, columns=t, index=t)
# Out puts a probability matrix (and Dataframe) that captures the win probability of the row team winning the column team

# OLD:
# for i in range(30):
#     win_probs = []
#     for j in range(30):
#         win_probs.append(choix.probabilities([i, j], skills)[0])
#     print(win_probs) # a probability matrix that captures the win probability of the row team winning the column team

In [28]:
prob_df.head()

,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,...,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Atlanta Hawks,0.500000,0.369730,0.773103,0.580707,0.521359,0.391213,0.606777,0.521968,0.567886,0.183792,...,0.426307,0.668825,0.682184,0.718258,0.507073,0.616447,0.255857,0.397675,0.380482,0.422176
Boston Celtics,0.630270,0.500000,0.853121,0.702462,0.649960,0.522774,0.724553,0.650515,0.691386,0.277381,...,0.558837,0.774911,0.785364,0.812937,0.636838,0.732603,0.369529,0.529519,0.511466,0.554663
Brooklyn Nets,0.226897,0.146879,0.500000,0.289001,0.242242,0.158674,0.311711,0.242690,0.278345,0.061990,...,0.179042,0.372142,0.386491,0.427984,0.231898,0.320512,0.091660,0.162319,0.152721,0.176570
Charlotte Hornets,0.419293,0.297538,0.710999,0.500000,0.440240,0.316935,0.527001,0.440842,0.486892,0.139849,...,0.349188,0.593198,0.607818,0.647977,0.426197,0.537136,0.198883,0.322821,0.307213,0.345355
Chicago Bulls,0.478641,0.350040,0.757758,0.559760,0.500000,0.371052,0.586204,0.500610,0.546798,0.171312,...,0.405543,0.649624,0.663368,0.700640,0.485705,0.596043,0.239923,0.377388,0.360547,0.401472


In [33]:
skills_norm = (skills - skills.min())/(skills.max()-skills.min())# fit between o and 1
ranking = pd.DataFrame(list(zip(t,list(skills_norm))), columns = ['team','score']).sort_values('score', ascending=False).reset_index(drop = True)
ranking
# seem reasnoble according to https://www.espn.com/nba/standings/_/season/2017
# not sure why I want to nor it.. Seems prudent though

,team,score
0,Golden State Warriors,1.000000
1,San Antonio Spurs,0.844208
2,Houston Rockets,0.715555
3,Boston Celtics,0.647564
4,Los Angeles Clippers,0.636289
5,Utah Jazz,0.630680
6,Cleveland Cavaliers,0.614010
7,Toronto Raptors,0.604052
8,Washington Wizards,0.566760
9,Oklahoma City Thunder,0.560534


Next up UFC light weight ;)